# Pengenalan kepada Kebarangkalian dan Statistik
Dalam buku nota ini, kita akan bermain-main dengan beberapa konsep yang telah kita bincangkan sebelum ini. Banyak konsep daripada kebarangkalian dan statistik diwakili dengan baik dalam perpustakaan utama untuk pemprosesan data dalam Python, seperti `numpy` dan `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Pembolehubah Rawak dan Taburan
Mari kita mulakan dengan mengambil sampel 30 nilai dari taburan seragam dari 0 hingga 9. Kita juga akan mengira min dan varians.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Untuk menganggar secara visual berapa banyak nilai berbeza yang terdapat dalam sampel, kita boleh plot **histogram**:


In [ ]:
plt.hist(sample)
plt.show()

## Menganalisis Data Sebenar

Min dan varians adalah sangat penting apabila menganalisis data dunia sebenar. Mari muatkan data tentang pemain besbol dari [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Kami menggunakan pakej yang dipanggil [**Pandas**](https://pandas.pydata.org/) di sini untuk analisis data. Kami akan bercakap lebih lanjut mengenai Pandas dan bekerja dengan data dalam Python kemudian dalam kursus ini.

Mari kita kira nilai purata untuk umur, ketinggian dan berat:


In [ ]:
df[['Age','Height','Weight']].mean()

Sekarang mari kita fokus pada ketinggian, dan kira sisihan piawai serta varians:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Selain daripada min, adalah munasabah untuk melihat nilai median dan kuartil. Ia boleh divisualisasikan menggunakan **graf kotak**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Kita juga boleh membuat plot kotak bagi subset dataset kita, contohnya, dikelompokkan mengikut peranan pemain.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Nota**: Rajah ini mencadangkan, secara purata, ketinggian pemain pertama lebih tinggi daripada ketinggian pemain kedua. Nanti kita akan belajar bagaimana kita boleh menguji hipotesis ini dengan lebih formal, dan bagaimana untuk menunjukkan bahawa data kita adalah signifikan secara statistik untuk membuktikannya.  

Umur, ketinggian dan berat semua adalah pembolehubah rawak berterusan. Apakah agaknya taburan mereka? Cara yang baik untuk mengetahuinya ialah dengan melukis histogram nilai-nilai tersebut: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Taburan Normal

Mari kita cipta sampel buatan berat yang mengikuti taburan normal dengan min dan varians yang sama seperti data sebenar kita:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Oleh kerana kebanyakan nilai dalam kehidupan sebenar diedarkan secara normal, kita tidak seharusnya menggunakan penjana nombor rawak seragam untuk menjana data sampel. Berikut adalah apa yang berlaku jika kita cuba menjana berat dengan taburan seragam (dijana oleh `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Selang Keyakinan

Mari kita kira selang keyakinan untuk berat dan ketinggian pemain besbol. Kita akan menggunakan kod [dari perbincangan stackoverflow ini](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Ujian Hipotesis

Mari kita terokai peranan berbeza dalam set data pemain besbol kami:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Mari uji hipotesis bahawa Pemain Pertama Bas lebih tinggi daripada Pemain Kedua Bas. Cara paling mudah untuk melakukan ini adalah dengan menguji selang keyakinan:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Kita dapat melihat bahawa julat masa tidak bertindih.

Cara yang lebih tepat dari segi statistik untuk membuktikan hipotesis adalah dengan menggunakan **ujian t-Student**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Dua nilai yang dikembalikan oleh fungsi `ttest_ind` adalah:
* nilai p boleh dianggap sebagai kebarangkalian dua taburan mempunyai min yang sama. Dalam kes kami, ia sangat rendah, bermakna terdapat bukti kukuh yang menyokong bahawa pemain pertama adalah lebih tinggi.
* nilai t adalah nilai pertengahan perbezaan min yang dinormalisasi yang digunakan dalam ujian t, dan ia dibandingkan dengan nilai ambang untuk nilai keyakinan tertentu.


## Mensimulasikan Taburan Normal dengan Teorem Had Tengah

Penjana pseudo-rawak dalam Python direka untuk memberikan taburan seragam. Jika kita ingin membuat penjana untuk taburan normal, kita boleh menggunakan teorem had tengah. Untuk mendapatkan nilai yang diedarkan secara normal, kita hanya akan mengira purata sampel yang dijana secara seragam.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korelasi dan Evil Baseball Corp

Korelasi membolehkan kita mencari hubungan antara urutan data. Dalam contoh mainan kita, mari kita andaikan terdapat sebuah syarikat besbol jahat yang membayar pemainnya mengikut ketinggian mereka - semakin tinggi pemain tersebut, semakin banyak wang yang dia terima. Katakan terdapat gaji asas sebanyak $1000, dan bonus tambahan dari $0 hingga $100, bergantung pada ketinggian. Kita akan mengambil pemain sebenar dari MLB, dan mengira gaji khayalan mereka:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Mari kita kira kovarians dan korelasi bagi urutan-urutan tersebut. `np.cov` akan memberikan kita yang dipanggil **matriks kovarians**, yang merupakan lanjutan kovarians kepada pelbagai pemboleh ubah. Elemen $M_{ij}$ dalam matriks kovarians $M$ adalah korelasi antara pemboleh ubah input $X_i$ dan $X_j$, dan nilai diagonal $M_{ii}$ adalah varians bagi $X_{i}$. Begitu juga, `np.corrcoef` akan memberikan kita **matriks korelasi**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Korelasi sama dengan 1 bermaksud terdapat **hubungan linear** yang kuat antara dua pemboleh ubah. Kita boleh lihat hubungan linear secara visual dengan memplot satu nilai terhadap yang lain:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Mari kita lihat apa yang berlaku jika hubungannya tidak linear. Katakan bahawa syarikat kita memutuskan untuk menyembunyikan pergantungan linear yang jelas antara ketinggian dan gaji, dan memperkenalkan sedikit bukan linear dalam formula, seperti `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Dalam kes ini, korelasi adalah sedikit lebih kecil, tetapi masih agak tinggi. Kini, untuk menjadikan hubungan itu kurang jelas, kita mungkin mahu menambah beberapa unsur rawak tambahan dengan menambah beberapa pembolehubah rawak kepada gaji. Mari lihat apa yang berlaku:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Bolehkah anda teka mengapa titik-titik itu tersusun menjadi garisan menegak seperti ini?

Kita telah memerhati korelasi antara konsep yang direka secara artifisial seperti gaji dan pemboleh ubah yang diperhatikan *tinggi*. Mari kita lihat juga jika dua pemboleh ubah yang diperhatikan, seperti tinggi dan berat, juga berkorelasi:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Malangnya, kami tidak mendapat sebarang keputusan - hanya beberapa nilai `nan` pelik. Ini kerana beberapa nilai dalam siri kami tidak ditakrifkan, yang diwakili sebagai `nan`, yang menyebabkan hasil operasi itu juga tidak ditakrifkan. Dengan melihat matriks, kita dapat lihat bahawa `Weight` adalah lajur bermasalah, kerana korelasi sendiri antara nilai `Height` telah dikira.

> Contoh ini menunjukkan kepentingan **penyediaan data** dan **pembersihan**. Tanpa data yang betul, kita tidak dapat mengira apa-apa.

Mari kita gunakan kaedah `fillna` untuk mengisi nilai yang hilang, dan mengira korelasi:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Memang terdapat korelasi, tetapi tidaklah sehebat dalam contoh buatan kami. Sebenarnya, jika kita melihat plot taburan satu nilai terhadap nilai yang lain, hubungannya akan menjadi kurang jelas:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Kesimpulan

Dalam buku nota ini kita telah belajar bagaimana untuk melaksanakan operasi asas ke atas data untuk mengira fungsi statistik. Kita kini tahu bagaimana menggunakan peralatan matematik dan statistik yang mantap untuk membuktikan beberapa hipotesis, dan bagaimana untuk mengira selang keyakinan bagi pemboleh ubah arbitrari berdasarkan sampel data.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk ketepatan, sila maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya hendaklah dianggap sebagai sumber yang sahih. Untuk maklumat penting, terjemahan oleh profesional manusia adalah disyorkan. Kami tidak bertanggungjawab terhadap sebarang salah faham atau kesilapan tafsir yang timbul daripada penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
